In [7]:
import dill
dill.dump_session("EXP1.db")

In [1]:
import dill 
dill.load_session("EXP1.db")

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split



In [2]:
from keras.models import Sequential
from keras.layers import Conv1D, ConvLSTM2D, MaxPooling1D, Flatten, Dense, Reshape,TimeDistributed,Conv2D,BatchNormalization,MaxPooling2D,ConvLSTM1D,LSTM



In [3]:
import tensorflow as tf
seed_value = 42
np.random.seed(seed_value)
tf.random.set_seed(seed_value)


In [4]:
# Read CSV File
df = pd.read_csv(r"C:\Users\mujta\OneDrive\Documents\GitHub\F20PA-DriverDrowsinessDetection\Experiments\mergedEMYA_min_max_normEMYA.csv")

In [5]:
# Select frames at 1 fps
df = df.iloc[::10]


In [6]:
df.drop(columns=["Unnamed: 0","Angle","Part_ID"],inplace=True)

In [7]:
df

,Label,EAR,MAR,MOE
0,0,0.695510,0.616864,0.391060
10,0,0.748416,0.650276,0.388583
20,0,0.777219,0.650798,0.376921
30,0,0.310910,0.590178,0.616396
40,0,0.677256,0.632891,0.409526
...,...,...,...,...
422950,10,0.173777,0.563920,0.663220
422960,10,0.296172,0.674049,0.558488
422970,10,0.440577,0.598430,0.399449
422980,10,0.197943,0.708829,0.690436


In [8]:
int(df.shape[0]/5)

8460

In [9]:

from sklearn.metrics import precision_recall_fscore_support
# Function to get precision, recall, and F1 scores
def getPerformanceMetrics(model,X_test,y_test):

    y_pred_proba = model.predict(X_test)
    y_pred = np.argmax(y_pred_proba, axis=1)
    
    # Compute precision, recall, and F1 score for each class
    precision, recall, f1, _ = precision_recall_fscore_support(y_test.flatten(), y_pred, average=None)
    # Lists of precision , recall, and F1 scores
    p_list = []
    r_list = []
    f_list = []
    # Print precision, recall, and F1 score for each class
    for i in range(len(precision)):
        p_list.append(precision[i])
        r_list.append(recall[i])
        f_list.append(f1[i])
        print(f"Class {i}:")
        print(f"  Precision: {precision[i]}")
        print(f"  Recall: {recall[i]}")
        print(f"  F1 Score: {f1[i]}")
    # Print Average values
    print(f"Avg Precision:{np.average(p_list)}")
    print(f"Avg Recall:{np.average(r_list)}")
    print(f"Avg F1 Score:{np.average(f_list)}")


In [10]:
# Function to set parameters and train model.
# Returns model, and test sets.
def compileModel(data,timestep,test_size,num_epochs):
    # Separate labels
    y=data["Label"]
    data = data.drop(columns=['Label'])
    # Reshape according to timestep
    df_reshaped = np.array(data).reshape(int(data.shape[0]/timestep),timestep,3)
    y = np.array(y)
    y_reshaped  = []
    for i in range(0, len(y), timestep):
        y_reshaped.append([y[i]])
    # Create test and  train sets.
    X_train, X_test, y_train, y_test = train_test_split(df_reshaped, y_reshaped, test_size=test_size, random_state=42)

    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train= np.array(y_train)
    y_test = np.array(y_test)
    # Set labels as  0,1,2
    for i in range(0,len(y_train)):
        y_train[i]=int(y_train[i]/5)
    for i in range(0,len(y_test)):
        y_test[i]=int(y_test[i]/5)
    # Define model
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(timestep, 3)))
    model.add(MaxPooling1D(pool_size=2))

    model.add(LSTM(64, return_sequences=True))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(3, activation='softmax'))  

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=num_epochs, batch_size=32, validation_split=0.2)

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test)
    print("Accuracy:", accuracy)
    getPerformanceMetrics(model,X_test,y_test)
    return model, X_test, y_test






## Increase timestep 5,10,20

In [11]:
modelA , X_testA, y_testA = compileModel(data=df,
                                         timestep=5,
                                         test_size=0.2,
                                         num_epochs=50)




Epoch 1/50


170/170 [==============================] - 4s 9ms/step - loss: 1.0857 - accuracy: 0.3953 - val_loss: 1.0644 - val_accuracy: 0.4084
Epoch 2/50
170/170 [==============================] - 1s 4ms/step - loss: 1.0631 - accuracy: 0.4213 - val_loss: 1.0520 - val_accuracy: 0.4247
Epoch 3/50
170/170 [==============================] - 1s 3ms/step - loss: 1.0504 - accuracy: 0.4320 - val_loss: 1.0468 - val_accuracy: 0.4313
Epoch 4/50
170/170 [==============================] - 1s 4ms/step - loss: 1.0440 - accuracy: 0.4381 - val_loss: 1.0361 - val_accuracy: 0.4276
Epoch 5/50
170/170 [==============================] - 1s 4ms/step - loss: 1.0397 - accuracy: 0.4357 - val_loss: 1.0353 - val_accuracy: 0.4217
Epoch 6/50
170/170 [==============================] - 1s 4ms/step - loss: 1.0342 - accuracy: 0.4440 - val_loss: 1.0291 - val_accuracy: 0.4202
Epoch 7/50
170/170 [==============================] - 1s 4ms/step - loss: 1.0320 - accuracy: 0.4368 - val_loss: 1.0287 - val_accuracy: 0.4298
E

In [12]:
modelA.save("modelA.h5")

c:\Users\mujta\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [13]:
modelB , X_testB, y_testB = compileModel(data=df,
                                         timestep=10,
                                         test_size=0.2,
                                         num_epochs=50)

Epoch 1/50
85/85 [==============================] - 5s 15ms/step - loss: 1.0852 - accuracy: 0.3986 - val_loss: 1.0666 - val_accuracy: 0.4357
Epoch 2/50
85/85 [==============================] - 1s 6ms/step - loss: 1.0614 - accuracy: 0.4141 - val_loss: 1.0588 - val_accuracy: 0.4357
Epoch 3/50
85/85 [==============================] - 0s 5ms/step - loss: 1.0457 - accuracy: 0.4230 - val_loss: 1.0531 - val_accuracy: 0.4269
Epoch 4/50
85/85 [==============================] - 1s 6ms/step - loss: 1.0355 - accuracy: 0.4296 - val_loss: 1.0668 - val_accuracy: 0.4062
Epoch 5/50
85/85 [==============================] - 1s 6ms/step - loss: 1.0418 - accuracy: 0.4374 - val_loss: 1.0477 - val_accuracy: 0.4195
Epoch 6/50
85/85 [==============================] - 0s 6ms/step - loss: 1.0234 - accuracy: 0.4588 - val_loss: 1.0471 - val_accuracy: 0.4151
Epoch 7/50
85/85 [==============================] - 1s 7ms/step - loss: 1.0212 - accuracy: 0.4492 - val_loss: 1.0429 - val_accuracy: 0.4357
Epoch 8/50
85/85 [=

In [14]:
modelB.save("modelB.h5")

c:\Users\mujta\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [15]:
modelC , X_testC, y_testC = compileModel(data=df,
                                         timestep=20,
                                         test_size=0.2,
                                         num_epochs=50)

Epoch 1/50
43/43 [==============================] - 5s 28ms/step - loss: 1.0908 - accuracy: 0.3710 - val_loss: 1.0832 - val_accuracy: 0.4071
Epoch 2/50
43/43 [==============================] - 0s 9ms/step - loss: 1.0731 - accuracy: 0.4028 - val_loss: 1.0548 - val_accuracy: 0.4690
Epoch 3/50
43/43 [==============================] - 0s 8ms/step - loss: 1.0561 - accuracy: 0.4464 - val_loss: 1.0366 - val_accuracy: 0.4690
Epoch 4/50
43/43 [==============================] - 0s 9ms/step - loss: 1.0466 - accuracy: 0.4339 - val_loss: 1.0250 - val_accuracy: 0.4336
Epoch 5/50
43/43 [==============================] - 0s 9ms/step - loss: 1.0504 - accuracy: 0.4457 - val_loss: 1.0079 - val_accuracy: 0.4720
Epoch 6/50
43/43 [==============================] - 0s 9ms/step - loss: 1.0321 - accuracy: 0.4708 - val_loss: 1.0271 - val_accuracy: 0.4425
Epoch 7/50
43/43 [==============================] - 0s 8ms/step - loss: 1.0259 - accuracy: 0.4494 - val_loss: 1.0257 - val_accuracy: 0.4631
Epoch 8/50
43/43 [=

In [16]:
modelC.save("modelC.h5")

c:\Users\mujta\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Since Model B has best performance use that to experiamnet with num epochs

In [2]:
modelD , X_testD, y_testD = compileModel(data=df,
                                         timestep=10,
                                         test_size=0.2,
                                         num_epochs=75)

Epoch 1/75


85/85 [==============================] - 3s 9ms/step - loss: 1.0828 - accuracy: 0.3905 - val_loss: 1.0829 - val_accuracy: 0.4328
Epoch 2/75
85/85 [==============================] - 0s 5ms/step - loss: 1.0564 - accuracy: 0.4270 - val_loss: 1.0553 - val_accuracy: 0.4520
Epoch 3/75
85/85 [==============================] - 0s 4ms/step - loss: 1.0420 - accuracy: 0.4433 - val_loss: 1.0566 - val_accuracy: 0.4461
Epoch 4/75
85/85 [==============================] - 0s 4ms/step - loss: 1.0377 - accuracy: 0.4381 - val_loss: 1.0540 - val_accuracy: 0.4343
Epoch 5/75
85/85 [==============================] - 0s 4ms/step - loss: 1.0315 - accuracy: 0.4463 - val_loss: 1.0463 - val_accuracy: 0.4357
Epoch 6/75
85/85 [==============================] - 0s 4ms/step - loss: 1.0203 - accuracy: 0.4573 - val_loss: 1.0462 - val_accuracy: 0.4476
Epoch 7/75
85/85 [==============================] - 0s 4ms/step - loss: 1.0186 - accuracy: 0.4588 - val_loss: 1.0400 - val_accuracy: 0.4298
Epoch 8/75
85/85 [

In [3]:
modelD.save("modelD.h5")

c:\Users\mujta\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [4]:
modelE , X_testE, y_testE = compileModel(data=df,
                                         timestep=10,
                                         test_size=0.2,
                                         num_epochs=100)

Epoch 1/100
85/85 [==============================] - 2s 9ms/step - loss: 1.0803 - accuracy: 0.3934 - val_loss: 1.0739 - val_accuracy: 0.3826
Epoch 2/100
85/85 [==============================] - 0s 4ms/step - loss: 1.0513 - accuracy: 0.4285 - val_loss: 1.0808 - val_accuracy: 0.3826
Epoch 3/100
85/85 [==============================] - 0s 4ms/step - loss: 1.0482 - accuracy: 0.4182 - val_loss: 1.0505 - val_accuracy: 0.4313
Epoch 4/100
85/85 [==============================] - 0s 4ms/step - loss: 1.0341 - accuracy: 0.4396 - val_loss: 1.0535 - val_accuracy: 0.4165
Epoch 5/100
85/85 [==============================] - 0s 4ms/step - loss: 1.0262 - accuracy: 0.4474 - val_loss: 1.0489 - val_accuracy: 0.4402
Epoch 6/100
85/85 [==============================] - 0s 4ms/step - loss: 1.0227 - accuracy: 0.4422 - val_loss: 1.0442 - val_accuracy: 0.4121
Epoch 7/100
85/85 [==============================] - 0s 4ms/step - loss: 1.0185 - accuracy: 0.4540 - val_loss: 1.0531 - val_accuracy: 0.3973
Epoch 8/100
8

In [5]:
modelE.save("modelE.h5")

c:\Users\mujta\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Use best performance and experiment with test size

### 70-30 split

In [6]:
modelF , X_testF, y_testF = compileModel(data=df,
                                         timestep=10,
                                         test_size=0.3,
                                         num_epochs=100)

Epoch 1/100
74/74 [==============================] - 2s 9ms/step - loss: 1.0897 - accuracy: 0.3775 - val_loss: 1.0805 - val_accuracy: 0.4384
Epoch 2/100
74/74 [==============================] - 0s 4ms/step - loss: 1.0662 - accuracy: 0.4177 - val_loss: 1.0712 - val_accuracy: 0.4570
Epoch 3/100
74/74 [==============================] - 0s 3ms/step - loss: 1.0518 - accuracy: 0.4286 - val_loss: 1.0757 - val_accuracy: 0.3744
Epoch 4/100
74/74 [==============================] - 0s 3ms/step - loss: 1.0435 - accuracy: 0.4375 - val_loss: 1.0709 - val_accuracy: 0.3879
Epoch 5/100
74/74 [==============================] - 0s 3ms/step - loss: 1.0386 - accuracy: 0.4468 - val_loss: 1.0606 - val_accuracy: 0.4216
Epoch 6/100
74/74 [==============================] - 0s 3ms/step - loss: 1.0299 - accuracy: 0.4468 - val_loss: 1.0882 - val_accuracy: 0.3997
Epoch 7/100
74/74 [==============================] - 0s 3ms/step - loss: 1.0389 - accuracy: 0.4350 - val_loss: 1.0752 - val_accuracy: 0.4216
Epoch 8/100
7

In [7]:
modelF.save("modelF.h5")

c:\Users\mujta\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### 50-50 split

In [8]:
modelG , X_testG, y_testG = compileModel(data=df,
                                         timestep=10,
                                         test_size=0.5,
                                         num_epochs=100)

Epoch 1/100
53/53 [==============================] - 2s 11ms/step - loss: 1.0904 - accuracy: 0.3741 - val_loss: 1.0795 - val_accuracy: 0.4681
Epoch 2/100
53/53 [==============================] - 0s 4ms/step - loss: 1.0730 - accuracy: 0.4125 - val_loss: 1.0696 - val_accuracy: 0.4350
Epoch 3/100
53/53 [==============================] - 0s 4ms/step - loss: 1.0621 - accuracy: 0.4403 - val_loss: 1.0740 - val_accuracy: 0.4090
Epoch 4/100
53/53 [==============================] - 0s 4ms/step - loss: 1.0503 - accuracy: 0.4356 - val_loss: 1.0717 - val_accuracy: 0.4113
Epoch 5/100
53/53 [==============================] - 0s 4ms/step - loss: 1.0498 - accuracy: 0.4486 - val_loss: 1.0581 - val_accuracy: 0.4374
Epoch 6/100
53/53 [==============================] - 0s 4ms/step - loss: 1.0415 - accuracy: 0.4521 - val_loss: 1.0565 - val_accuracy: 0.4137
Epoch 7/100
53/53 [==============================] - 0s 4ms/step - loss: 1.0367 - accuracy: 0.4710 - val_loss: 1.0487 - val_accuracy: 0.4303
Epoch 8/100


In [9]:
modelG.save("modelG.h5")

c:\Users\mujta\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
